In [1]:
import warnings

# import holoviews as hv
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import numpy as np
import shapely
import pandas as pd
import pystac_client
import xarray as xr
import rioxarray as rio
import pathlib

#import colormaps as cmaps
import pyam # https://pyam-iamc.readthedocs.io/en/latest/index.html
import matplotlib.colors as mcolors

from copy import deepcopy
from typing import List, Dict

# ignore warnings
warnings.filterwarnings("ignore")

In [5]:
# load the CoCliCo STAC catalog
catalog = pystac_client.Client.open(
    "https://storage.googleapis.com/coclico-data-public/coclico/coclico-stac/catalog.json"
)
# catalog

# list the datasets present in the catalog, we are interested in the slp5 and slp6 sets
list(catalog.get_children())

[<CollectionClient id=floodmaps>,
 <CollectionClient id=cbca>,
 <CollectionClient id=cfr>,
 <CollectionClient id=smd>,
 <CollectionClient id=cisi>,
 <CollectionClient id=slp5>,
 <CollectionClient id=slp6>,
 <CollectionClient id=coastal-mask>,
 <CollectionClient id=overture-building-footprints>,
 <CollectionClient id=gctr>,
 <CollectionClient id=ceed>,
 <CollectionClient id=shorelinemonitor-shorelines>,
 <CollectionClient id=shorelinemonitor-raw-series>,
 <CollectionClient id=gcts>,
 <CollectionClient id=slp>,
 <CollectionClient id=ss_wc>,
 <CollectionClient id=twl>,
 <CollectionClient id=cet>,
 <CollectionClient id=CFHP_LAU_stats>,
 <CollectionClient id=ssl>,
 <CollectionClient id=wef>,
 <CollectionClient id=eesl>,
 <CollectionClient id=sc>,
 <CollectionClient id=NUTS_CM>,
 <CollectionClient id=NUTS>,
 <CollectionClient id=LAU>,
 <CollectionClient id=deltares-delta-dtm>,
 <CollectionClient id=pp>,
 <CollectionClient id=LAU_CM>,
 <CollectionClient id=NUTS0_CM>,
 <CollectionClient id=NUT

In [6]:
# function to put items in dataframe
def items_to_dataframe(items: List[Dict]) -> pd.DataFrame:
    """STAC items to Pandas dataframe.

    Args:
        items (List[Dict]): _description_

    Returns:
        pd.DataFrame: _description_
    """
    _items = []
    for i in items:
        _i = deepcopy(i)
        # _i['geometry'] = shape(_i['geometry'])
        # ...  # for example, drop some attributes that you're not interested in
        _items.append(_i)
    df = pd.DataFrame(pd.json_normalize(_items))
    # for field in ["properties.datetime"]:
    #     if field in df:
    #         df[field] = pd.to_datetime(df[field])
    # df = df.sort_values("properties.datetime")
    return df

In [7]:
ar5_col = catalog.get_child("cba_stats")
items_ar5 = list(ar5_col.get_items()) # this is slow as we need to list all items
items_ar5_df = items_to_dataframe([i.to_dict() for i in items_ar5])

In [8]:
# filter AR5 dataframe and STAC items on index
items_df_fil5 = items_ar5_df.filter(items = [0], axis=0)
items_fil5 = [items_ar5[i] for i in [0]]

In [ ]:
ar5_item_href = items_fil5[0].assets["data"].href

ar5_item = rio.open_rasterio(ar5_item_href, masked=True)